# Design Patterns with First-Class Functions

**In software engineering, a design pattern is a general recipe for solving a common design problem.**

The use of design patterns in programming was popularized by the landmark book Design Patterns: Elements of Reusable Object-Oriented Software (Addison-Wesley) by Erich Gamma, Richard Helm, Ralph Johnson, and John Vlissides—a.k.a. “the Gang of Four.” The book is a catalog of 23 patterns consisting of arrangements of classes exemplified with code in C++, but assumed to be useful in other object-oriented languages as well.

**Although design patterns are language independent, that does not mean every pattern applies to every language.**

The authors of Design Patterns acknowledge in their introduction that the implementation language determines which patterns are relevant:

> The choice of programming language is important because it influences one’s point of view. Our patterns assume Smalltalk/C++-level language features, and that choice determines what can and cannot be implemented easily. If we assumed procedural languages, we might have included design patterns called “Inheritance,” “Encapsulation,” and “Polymorphism.” Similarly, some of our patterns are supported directly by the less common object-oriented languages. CLOS has multi-methods, for example, which lessen the need for a pattern such as Visitor.

In his 1996 presentation, “Design Patterns in Dynamic Languages”, Peter Norvig states that 16 out of the 23 patterns in the original Design Patterns book become either “invisible or simpler” in a dynamic language (slide 9). He’s talking about the Lisp and Dylan languages, but many of the relevant dynamic features are also present in Python. In particular, in the context of languages with first-class functions, Norvig suggests rethinking the classic patterns known as Strategy, Command, Template Method, and Visitor.

**In some cases functions can do the same work as classes, with code that is more readable and concise.**

## Case Study: Refactoring Strategy

Strategy is a good example of a design pattern that can be simpler in Python if you leverage functions as first-class objects. In the following section, we describe and implement Strategy using the “classic” structure described in Design Patterns. If you are familiar with the classic pattern, you can skip to “Function-Oriented Strategy” where we refactor the code using functions, significantly reducing the line count.

### Classic Strategy

The Strategy pattern is summarized like this in Design Patterns:
- **Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary independently from clients that use it.**

A clear example of Strategy applied in the ecommerce domain is computing discounts to orders according to the attributes of the customer or inspection of the ordered items.

Consider an online store with these discount rules:
- Customers with 1,000 or more fidelity points get a global 5% discount per order.
- A 10% discount is applied to each line item with 20 or more units in the same order.
- Orders with at least 10 distinct items get a 7% global discount.

For brevity, let’s assume that only one discount may be applied to an order.

The UML class diagram for the Strategy pattern is depicted in Figure 10-1. Its participants are:

<img src="images/img1.png" height="100"/>

- **Context**: Provides a service by delegating some computation to interchangeable components that implement alternative algorithms. In the ecommerce example, the context is an Order, which is configured to apply a promotional discount according to one of several algorithms.
- **Strategy**: The interface common to the components that implement the different algorithms. In our example, this role is played by an abstract class called Promotion.
- **Concrete strategy**: One of the concrete subclasses of Strategy. FidelityPromo, BulkPromo, and LargeOrderPromo are the three concrete strategies implemented.

The code in Example 10-1 follows the blueprint in Figure 10-1. As described in Design Patterns, the concrete strategy is chosen by the client of the context class. In our example, before instantiating an order, the system would somehow select a promotional discount strategy and pass it to the Order constructor. The selection of the strategy is outside the scope of the pattern.

In [1]:
# 01_primer.py
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal
        
    def total(self) -> Decimal:
        return self.price * self.quantity
    
class Order(NamedTuple):  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'


class Promotion(ABC):  # the Strategy: an abstract base class
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)

Note that in Example 10-1, I coded Promotion as an abstract base class (ABC) to use the @abstractmethod decorator and make the pattern more explicit.

Shows doctests used to demonstrate and verify the operation of a module implementing the rules described earlier.

In [2]:
joe = Customer('John Doe', 0)

In [3]:
ann = Customer('Ann Smith', 1100)
# Two customers: joe has 0 fidelity points, ann has 1,100

In [4]:
# One shopping cart with three line items
cart = (LineItem('banana', 4, Decimal('.5')),  
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5)))

In [5]:
# The FidelityPromo promotion gives no discount to joe.
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [7]:
# ann gets a 5% discount because she has at least 1,000 points
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [8]:
# The banana_cart has 30 units of the "banana" product and 10 apples.
banana_cart = (LineItem('banana', 30, Decimal('.5')),  
               LineItem('apple', 10, Decimal('1.5')))

In [9]:
# Thanks to the BulkItemPromo, joe gets a $1.50 discount on the bananas.
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [11]:
# long_cart has 10 different items at $1.00 each
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) for sku in range(10))

In [12]:
# joe gets a 7% discount on the whole order because of LargerOrderPromo.
Order(joe, long_cart, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [13]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

Example 10-1 works perfectly well, but the same functionality can be implemented with less code in Python by using functions as objects. The next section shows how.

### Function-Oriented Strategy

Each concrete strategy in Example 10-1 is a class with a single method, discount. Furthermore, **the strategy instances have no state (no instance attributes). You could say they look a lot like plain functions, and you would be right**. Example 10-3 is a refactoring of Example 10-1, replacing the concrete strategies with simple functions and removing the Promo abstract class. Only small adjustments are needed in the Order class.

In [16]:
from collections.abc import Sequence
from dataclasses import dataclass
from decimal import Decimal
from typing import Optional, Callable, NamedTuple


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self):
        return self.price * self.quantity


@dataclass(frozen=True)
class Order:  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    # This type hint says: promotion may be None, or it may be a callable 
    # that takes an Order argument and returns a Decimal.
    promotion: Optional[Callable[["Order"], Decimal]] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            # To compute a discount, call the self.promotion callable, passing self as 
            # an argument. See the following tip for the reason.
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        return f"<Order total: {self.total():.2f} due: {self.due():.2f}>"


def fidelity_promo(order: Order) -> Decimal: # Each strategy is a function
    """5% discount for customers with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal("0.05")
    return Decimal(0)


def bulk_item_promo(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal("0.1")
    return discount


def large_order_promo(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal("0.07")
    return Decimal(0)


In the Order class, promotion is not a method. It’s an instance attribute that happens to be callable. So the first part of the expression, `self.promotion`, retrieves that callable. To invoke it, we must provide an instance of Order, which in this case is `self`. That’s why self appears twice in that expression.


The code in Example 10-3 is shorter than Example 10-1. Using the new Order is also a bit simpler, as shown in the Example 10-4 doctests.

In [17]:
joe = Customer('John Doe', 0)  
ann = Customer('Ann Smith', 1100)

cart = [LineItem('banana', 4, Decimal('.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5))]

In [18]:
# To apply a discount strategy to an Order, just pass the promotion function as an argument
Order(joe, cart, fidelity_promo)  

<Order total: 42.00 due: 42.00>

In [19]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [20]:
banana_cart = [LineItem('banana', 30, Decimal('.5')),
              LineItem('apple', 10, Decimal('1.5'))]

In [21]:
# A different promotion function is used here and in the next test.
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [22]:
long_cart = [LineItem(str(item_code), 1, Decimal(1)) for item_code in range(10)]

In [23]:
Order(joe, long_cart, large_order_promo)

<Order total: 10.00 due: 9.30>

In [24]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

Note the callouts in Example 10-4—there is no need to instantiate a new promotion object with each new order: the functions are ready to use.

It is interesting to note that in Design Patterns, the authors suggest: “Strategy objects often make good flyweights.” A definition of the Flyweight pattern in another part of that work states: “A flyweight is a shared object that can be used in multiple contexts simultaneously.” The sharing is recommended to reduce the cost of creating a new concrete strategy object when the same strategy is applied over and over again with every new context—with every new Order instance, in our example. So, to overcome a drawback of the Strategy pattern—its runtime cost—the authors recommend applying yet another pattern. Meanwhile, the line count and maintenance cost of your code are piling up.

A thornier use case, with complex concrete strategies holding internal state, may require all the pieces of the Strategy and Flyweight design patterns combined. But often concrete strategies have no internal state; they only deal with data from the context. If that is the case, then by all means use plain old functions instead of coding single-method classes implementing a single-method interface declared in yet another class. **A function is more lightweight than an instance of a user-defined class**, and there is no need for Flyweight because **each strategy function is created just once per Python process when it loads the module**. A plain function is also “a shared object that can be used in multiple contexts simultaneously.”

Now that we have implemented the Strategy pattern with functions, other possibilities emerge. Suppose you want to create a “metastrategy” that selects the best available discount for a given Order. In the following sections we study additional refactorings that implement this requirement using a variety of approaches that leverage functions and modules as objects.

### Choosing the Best Strategy: Simple Approach
